In [1]:
import numpy as np
import pandas as pd
import os.path
import matplotlib.pyplot as plt
from matplotlib import dates
%matplotlib inline
import time
import datetime as dt
import pandas_datareader.data as data
import scipy.cluster.hierarchy as sch
from scipy.optimize import minimize
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import scipy as sp
import pandas_datareader.data as web
from scipy.optimize import minimize
from os import walk
import datetime as dt
import pandas_datareader.data as data
import matplotlib.lines as mlines
from bokeh.models import ColumnDataSource
from scipy.optimize import minimize
from bokeh.plotting import output_file, show, figure
from bokeh.layouts import row
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool
from bokeh.models import DatetimeTickFormatter,Range1d, LabelSet, Label, BoxAnnotation,BoxSelectTool, BoxZoomTool, LassoSelectTool
from ipywidgets import interact
output_notebook()

E:\pythonana\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Loading BokehJS ...

Read Data

In [7]:
def extractPast(today):
    #extract past 3 month data before input date.
    count = 0
    total = period+2
    now = today
    path = 'F:/orie5280/data new/data/'
    d={}
    while count <= total:
        s = path + str(now) + '.csv'
        if os.path.exists(s):
            df = pd.read_csv(s)
            d[now] = df
            count += 1
        now = now - 1
    return d

Calculate Turnover Rate

In [8]:
def vol_pct_change(dataf,length):
    
    pct_top = []
    pct_mid = []
    for i in range(0,length):
        print(i)
        a,b = newTopVol(dataf,lst[i])
        c,d = newTopVol(dataf,lst[i+1])
        if len(list(set(a['cusip']).intersection(c['cusip'])))/a.shape[0] <= 0.5:
            print(lst[i])
            print(lst[i+1])
            print("--------")
        print(len(list(set(a['cusip']).intersection(c['cusip'])))/a.shape[0])
        pct_top.append(len(list(set(a['cusip']).intersection(c['cusip'])))/a.shape[0])
        #pct_mid.append(len(list(set(b['cusip']).intersection(d['cusip'])))/b.shape[0])
    return pct_top


Calculate Universe

In [9]:
def newTopVol(d,today,nextdf,nexttoday):
    count = 0
    ls = []
    zz = pd.DataFrame()
    result = pd.DataFrame()
    op = list(d.keys())
    for i in range(op.index(today),op.index(today)+5,1):
        zz = pd.concat([zz,d[op[i]]])  
    ls = zz.iloc[:,1]
    #-get next universe
    nextzz = pd.DataFrame()
    nextls = []
    nextop = list(nextdf.keys())
    for keys in nextdf:
        nextzz = pd.concat([nextzz,nextdf[keys]])  
    nextls = nextzz['cusip'].unique()
    #nextls = zz.iloc[:,1]
    #--
    zz = pd.DataFrame()
    for key in d:
        count += 1
        nowDf = d[key]
        nowDf = nowDf[nowDf['cusip'].isin(nextls)]
        nowDf = nowDf[nowDf['cusip'].isin(cusipLst)]
        #print(nowDf.shape)
        overlap = nowDf[nowDf['cusip'].isin(ls)].iloc[:,[0,1,2,5,6,7,9,17]]
        overlap = overlap[overlap['prccd']>=0.7].dropna()
        overlap = overlap[overlap['return']<=1.0].dropna()
        result = pd.concat([result,overlap])
        if count == period:
            break
    result['value'] = result["market_cap"]
    result['R'] = (result['prccd']-result['prev_prccd'])/result['prev_prccd']
    df = result.groupby('cusip').mean()
    df['vrank'] = df['value'].rank(ascending=False)
    df['mom'] = df['R'].rank(ascending=False)
    mom = df.sort_values(['mom'],ascending=True).head(int(0.3*n)).tail(int(0.3*n))
    val = df.sort_values(['vrank'],ascending=True).head(int(0.7*n)).tail(int(0.7*n))
    df = pd.concat([mom,val])
    
    #df['tot']=0.03*df['mom']+df['vrank']
    #df = df.sort_values(['tot'],ascending=True).head(n)
    
    
   # df = result.groupby('cusip').mean().sort_values(['value'], ascending=False).head(200)
    df['cusip'] = df.index
    return df

Filter Data

In [4]:
n = 100
period = 69
mypath = 'F:/orie5280/data new/data/'
for (dirpath, dirnames, filenames) in walk(mypath):
    print(filenames[:-4])
alldate = list(map(lambda x: x[:-4],filenames))
backtestD = list(filter(lambda x: int(x)>=20150101,alldate))
merg = pd.read_csv('trans_cost.csv')
cusipLst = merg['CUSIP']
cusipLst = list(map(lambda x: x[:-4],cusipLst))
merg['CUSIP'] = cusipLst
rebalanceD = backtestD[0::period]
rebalanceD=rebalanceD[0:]


['19831230.csv', '19840102.csv', '19840103.csv', '19840104.csv', '19840105.csv', '19840106.csv', '19840109.csv', '19840110.csv', '19840111.csv', '19840112.csv', '19840113.csv', '19840116.csv', '19840117.csv', '19840118.csv', '19840119.csv', '19840120.csv', '19840123.csv', '19840124.csv', '19840125.csv', '19840126.csv', '19840127.csv', '19840130.csv', '19840131.csv', '19840201.csv', '19840202.csv', '19840203.csv', '19840206.csv', '19840207.csv', '19840208.csv', '19840209.csv', '19840210.csv', '19840213.csv', '19840214.csv', '19840215.csv', '19840216.csv', '19840217.csv', '19840220.csv', '19840221.csv', '19840222.csv', '19840223.csv', '19840224.csv', '19840227.csv', '19840228.csv', '19840229.csv', '19840301.csv', '19840302.csv', '19840305.csv', '19840306.csv', '19840307.csv', '19840308.csv', '19840309.csv', '19840312.csv', '19840313.csv', '19840314.csv', '19840315.csv', '19840316.csv', '19840319.csv', '19840320.csv', '19840321.csv', '19840322.csv', '19840323.csv', '19840326.csv', '198403

Check Overlap

In [10]:
pct_top = []
tradeU = []
for i in range(0,len(rebalanceD)-2):
   # print(rebalanceD[i])
    print(i/len(rebalanceD))
    df1 = extractPast(int(rebalanceD[i]))
    
    #print(trade)
    df2 = extractPast(int(rebalanceD[i+1]))
    df3 = extractPast(int(rebalanceD[i+2]))
    pre = newTopVol(df1,int(rebalanceD[i]),df2,int(rebalanceD[i+1]))
    tradeU.append(list(pre['cusip']))
    nex = newTopVol(df2,int(rebalanceD[i+1]),df3,int(rebalanceD[i+2]))
    pct_top.append(len(list(set(pre['cusip']).intersection(nex['cusip'])))/n)
    print(pct_top[i])
    print('--------')
    

0.0
0.7
--------
0.08333333333333333
0.68
--------
0.16666666666666666
0.68
--------
0.25
0.69
--------
0.3333333333333333
0.68
--------
0.4166666666666667
0.73
--------
0.5
0.75
--------
0.5833333333333334
0.7
--------
0.6666666666666666
0.64
--------
0.75
0.67
--------


Universe Select

In [33]:
len(rebalanceD)-1

11

In [11]:
pct_top = []
tradeU = []
for i in range(0,len(rebalanceD)-1):
   # print(rebalanceD[i])
    print(i/len(rebalanceD))
    df1 = extractPast(int(rebalanceD[i]))
    
    print(i)
    df2 = extractPast(int(rebalanceD[i+1]))
    #df3 = extractPast(int(rebalanceD[i+2]))
    pre = newTopVol(df1,int(rebalanceD[i]),df2,int(rebalanceD[i+1]))
    tradeU.append(list(pre['cusip']))
    #nex = newTopVol(df2,int(rebalanceD[i+1]),df3,int(rebalanceD[i+2]))
    #pct_top.append(len(list(set(pre['cusip']).intersection(nex['cusip'])))/200)
    #print(pct_top[i])
    print('--------')
    

0.0
0
--------
0.08333333333333333
1
--------
0.16666666666666666
2
--------
0.25
3
--------
0.3333333333333333
4
--------
0.4166666666666667
5
--------
0.5
6
--------
0.5833333333333334
7
--------
0.6666666666666666
8
--------
0.75
9
--------
0.8333333333333334
10
--------


In [15]:
def ivp(cov):
    ivp = 1./np.diag(cov)
    ivp /= ivp.sum()
    return ivp

In [16]:
def hrp(cov):
    corr = np.divide(np.divide(cov, np.sqrt(np.diag(cov))).T, np.sqrt(np.diag(cov)))

    def getQuasiDiag(link):
        # Sort clustered items by distance
        link = link.astype(int)
        sortIx = pd.Series([link[-1, 0], link[-1, 1]])
        numItems = link[-1, 3]  # number of original items
        while sortIx.max() >= numItems:
            sortIx.index = range(0, sortIx.shape[0] * 2, 2)  # make space
            df0 = sortIx[sortIx >= numItems]  # find clusters
            i = df0.index
            j = df0.values - numItems
            sortIx[i] = link[j, 0]  # item 1
            df0 = pd.Series(link[j, 1], index=i + 1)
            sortIx = sortIx.append(df0)  # item 2
            sortIx = sortIx.sort_index()  # re-sort
            sortIx.index = range(sortIx.shape[0])  # re-index

        return sortIx.tolist()

    def getRecBipart(cov, sortIx):
        def getClusterVar(cov,cItems):
            # Compute variance per cluster
            cov_=cov.loc[cItems,cItems] # matrix slice
            w_=ivp(cov_).reshape(-1,1)
            cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0]
            return cVar

        # Compute HRP alloc
        w = pd.Series(1, index=sortIx)
        cItems = [sortIx]  # initialize all items in one cluster
        while len(cItems) > 0:
            cItems = [i[j:k] for i in cItems for j, k in ((0, int(len(i) / 2)),
                                                          (int(len(i) / 2), len(i))) if len(i) > 1]  # bi-section
            for i in range(0, len(cItems), 2):  # parse in pairs
                cItems0 = cItems[i]  # cluster 1
                cItems1 = cItems[i + 1]  # cluster 2
                cVar0 = getClusterVar(cov, cItems0)
                cVar1 = getClusterVar(cov, cItems1)
                alpha = 1 - cVar0 / (cVar0 + cVar1)
                w[cItems0] *= alpha  # weight 1
                w[cItems1] *= 1 - alpha  # weight 2
        return w

    def correlDist(corr):
        # A distance matrix based on correlation, where 0<=d[i,j]<=1
        # This is a proper distance metric
        dist = ((1 - corr) / 2.)**.5  # distance matrix
        return dist
    
    # Construct a hierarchical portfolio
    corr, cov = pd.DataFrame(corr), pd.DataFrame(cov)
    dist = correlDist(corr).fillna(0)
    link = sch.linkage(dist, 'single')
    sortIx = getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist()
    # recover labels
    hrp = getRecBipart(cov, sortIx)

    return hrp.sort_index()

In [39]:
def allin(Tcost, model):
    final = pd.DataFrame()
    final_lst = []
    value = 1000000
    day = []
    total_train = pd.DataFrame()
    for i in range(0,len(rebalanceD)-1):

        df1 = extractPast(int(rebalanceD[i]))
        df2 = extractPast(int(rebalanceD[i+1]))
        agg = pd.DataFrame()
        train = pd.DataFrame()
        for keys in df1:
            train_temp = df1[keys]
            #print(train_temp.shape)
            train_temp = train_temp[train_temp['cusip'].isin(tradeU[i])]
            #print(train_temp.shape)
            train_temp['date']=keys
            train = pd.concat([train,train_temp])
            #print(train.shape)
        for keys in df2:
            temp = df2[keys]
            temp = temp[temp['cusip'].isin(tradeU[i])]
            temp['date']=keys
            agg = pd.concat([agg,temp])
            #print(agg.shape)
        priagg = agg.groupby(['date','cusip'])['prccd'].first().unstack().fillna(method='ffill').fillna(method='bfill')    
        agg = agg.groupby(['date','cusip'])['return'].first().unstack().fillna(method='ffill').fillna(method='bfill')
        train = train.groupby(['date','cusip'])['return'].first().unstack().fillna(method='ffill').fillna(method='bfill')    
        total_train = pd.concat([total_train,train])
        #priagg = agg.pivot(index='date',columns='cusip',values='prccd')
        #agg = agg.pivot(index='date',columns='cusip',values='return').fillna(method='ffill').fillna(method='bfill')
        #train = train.pivot(index='date',columns='cusip',values='return').fillna(method='ffill').fillna(method='bfill')
        #-backtest-
        mu = train.mean()
        ann_cov1_skip = np.cov(train.T)
        if model =='MVO':
            bnd = ((-0.000,0.15),)*n
            w01 = np.ones((n, 1))/n
            lambd = (6.8)/2
            cons = (
                     {'type':'eq','fun': lambda x: np.sum(x)-1},
                     )
            def risk1_fullsample(w): return float(lambd * np.dot(w.T,np.dot(ann_cov1_skip, w))-np.dot(mu.T,w))
            fullsample_weights = minimize(risk1_fullsample, w01, method='SLSQP', bounds=bnd,
            constraints=cons).x
        if model =='IVP':
            fullsample_weights = ivp(ann_cov1_skip)
        if model =='HRP':
            fullsample_weights = np.array(hrp(ann_cov1_skip))
        jjindex = agg.index
        we = (fullsample_weights*value).reshape(1,n)
        we = we.repeat(agg.shape[0],axis=0)
        money = pd.DataFrame(we,index = jjindex)
        money.columns = [list(agg.columns)]
        allprice = priagg.copy()
        priagg.iloc[1:,:]=np.nan
        priagg=priagg.fillna(method='ffill')
        if Tcost:
            merg_model = merg[merg['CUSIP'].isin(priagg.columns)]
            merg_model = merg_model.iloc[:,1:3].reset_index()
            del merg_model['index']
            #merg.unstack(level=0)
            merg_model['t']=merg_model.index
            #one value in each row. As index is just 0,1,2,3,4...
            merg_model =merg_model.groupby(['t','CUSIP'])['avgspread'].first().unstack().fillna(method='ffill').fillna(method='bfill').iloc[0:1,:]
            merg_model = pd.concat([merg_model]*priagg.shape[0])
            merg_model.index = priagg.index
            #end T cost
            priagg = priagg+merg_model
        else:
            priagg = priagg
        position = money/priagg
        combine = allprice*position
        combine['sum'] = combine.sum(axis=1)
        #final = pd.concat([final,combine])
        #-
        value = combine.iloc[-1,-1]
        day.append(list(combine.index))
        final_lst.append(list(combine.iloc[:,-1]))
        print(i)
        print(int(rebalanceD[i+1]))
        print(value)
        dayLst = [item for items in day for item in items]
        newlist = [item for items in final_lst for item in items]
        ddd = pd.DataFrame(newlist,index = dayLst)
        ret_rs = ddd.pct_change().dropna()
        ret_rs.columns=['ret']
        ddd.columns = ['ret']
        ddd['Date'] = ddd.index
        ddd.loc[:, 'Date'] = ddd['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
        
    return ddd

In [ ]:
modelLst = ['MVO','IVP','HRP']
allin(Tcost=True,model=modelLst[0])

Plot PNL

In [50]:
modelLst = ['MVO','IVP','HRP']
r = []
for i in modelLst:
    print(i)
    r.append(allin(False,i))

MVO


E:\pythonana\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\pythonana\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
20150414
1206540.99785
1
20150722
1313177.36679
2
20151028
1004454.32294
3
20160208
812757.736573
4
20160517
862007.017265
5
20160824
1296280.72664
6
20161201
1212280.12047
7
20170314
1686115.43573
8
20170621
1682543.73912
9
20170928
1802247.50447
10
20180108
2373830.89907
IVP
0
20150414
1016556.24436
1
20150722
1005035.98725
2
20151028
974373.586347
3
20160208
870108.820681
4
20160517
915663.764766
5
20160824
963463.220154
6
20161201
935558.368505
7
20170314
987542.118369
8
20170621
1021487.47247
9
20170928
1045029.94227
10
20180108
1121659.44904
HRP
0
20150414
1042502.84109
1
20150722
1034750.41374
2
20151028
991980.457657
3
20160208
893700.77561
4
20160517
929648.330902
5
20160824
981012.301082
6
20161201
931687.53967
7
20170314
994211.11913
8
20170621
1030067.95593
9
20170928
1047209.90529
10
20180108
1136152.06645


In [52]:
ddd = r[0]
ret_rs = ddd.iloc[:,0:1].pct_change().dropna()
ds = ColumnDataSource(r[0])
SR = str(np.round((ret_rs['ret'].mean()*252)/(ret_rs['ret'].std()*np.sqrt(252)),decimals=4))
name = "Portfolio. Mean Return: "+ str(np.round(ret_rs['ret'].mean()*252,decimals=4)*100)+"%" + " | Std: " + str(np.round(ret_rs['ret'].std()*np.sqrt(252),decimals=4)*100)+"%"+" | SR:"+str(SR)

p = figure(title=(name)) 

p.plot_height=500
p.plot_width=950
p.line(source=ds, x='Date', y='ret')

p.xaxis.formatter=DatetimeTickFormatter(
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

low_box = BoxAnnotation(top=ddd['ret'].quantile(0.25), fill_alpha=0.1, fill_color='red')
mid_box = BoxAnnotation(bottom=ddd['ret'].quantile(0.25), top=ddd['ret'].quantile(0.75), fill_alpha=0.1, fill_color='green')
high_box = BoxAnnotation(bottom=ddd['ret'].quantile(0.75), fill_alpha=0.1, fill_color='red')

p.add_layout(low_box)
p.add_layout(mid_box)
p.add_layout(high_box)
cr = p.cross(source=ds, x='Date', y='ret', size=20,
                fill_color="blue", hover_fill_color="firebrick",
                fill_alpha=0.05, hover_alpha=0.3,
                line_color=None, hover_line_color="red")
p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='hline'))

show(p)

In [51]:
#ret_rs = ddd.pct_change().dropna()
ds = ColumnDataSource(r[0])
ds2 = ColumnDataSource(r[1])
ds3 = ColumnDataSource(r[2])
name = "Portfolio Comparison"

p = figure(title=(name)) 

p.plot_height=500
p.plot_width=950
p.line(source=ds, x='Date', y='ret',color='green',legend='MVO')
p.line(source=ds2, x='Date', y='ret',color='red',legend='IVP')
p.line(source=ds3, x='Date', y='ret',color='blue',legend='HRP')
p.xaxis.formatter=DatetimeTickFormatter(
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

show(p)